In [ ]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import copy
import IPython

datos_boya = pd.read_excel('./path_to_data.xlsx') #Read first sheet data


#TRAIN FLAG 0=load 1=train new model
TRAIN = 0

# Drop no valid data
datos_boya_no_zero = datos_boya.replace(0,np.nan)
datos_boya_proc = datos_boya_no_zero.dropna(subset=["T","DO","PH","FC_IVF"], how="all").reset_index(drop=True)
datos_boya_proc

In [ ]:
# Define arrays for the NN input
in0_d = (pd.DatetimeIndex(datos_boya_proc["date"]).month).values.reshape((len(datos_boya_proc["date"]),1))
in2_T = datos_boya_proc["T"].values.reshape((len(datos_boya_proc["T"]),1))
in5_DO = datos_boya_proc["DO"].values.reshape((len(datos_boya_proc["DO"]),1))
in6_PH = datos_boya_proc["PH"].values.reshape((len(datos_boya_proc["PH"]),1))
in10_FC = datos_boya_proc["FC_IVF"].values.reshape((len(datos_boya_proc["FC_IVF"]),1))

# Only selected parameters
real_dataset = np.hstack((in0_d, in2_T, in5_DO, in6_PH, in10_FC))


print("The shape of the dataset is")
np.shape(real_dataset)

In [ ]:
# Create full time series for the 5 years
# First extract total time
start_time = datos_boya_proc["date"][0]
end_time = datos_boya_proc["date"][np.shape(real_dataset)[0]-1]

# Compute number of 5 min time steps within these two dates
total_time_steps = int((end_time-start_time).total_seconds() / 60 / 5)

# Create empty full time series array
full_dataset = np.empty((int(total_time_steps), np.shape(real_dataset)[1]))
full_dataset[:]=np.nan

np.shape(full_dataset)

In [ ]:
# Fill full_dataset with real values. Create a list with each point being 5 min and add the real values to the corresponding position.
full_dataset[0]=real_dataset[0]; full_data_index=0; i=1; loosed_minutes=0
while(i<np.shape(real_dataset)[0]):
    # Get num time slots (5min) between the two values
    time_diff = (datos_boya_proc["date"][i] - datos_boya_proc["date"][i-1]).total_seconds()
    #print(loosed_minutes)
    time_jump = (time_diff+loosed_minutes*60)/60/5
    loosed_minutes = (time_diff+loosed_minutes*60)/60%5
    #print(i,time_jump, loosed_minutes, time_diff,(time_diff+loosed_minutes*60))
    
    # If number of slots is 0 it means data with a bad label, deal with it
    if(time_jump==0):
        num_same_values = 0
        # Advance until no data with same label
        while(((datos_boya_proc["date"][i] - datos_boya_proc["date"][i-1]).total_seconds()/60/5)==0):
            i+=1
            num_same_values +=1
            
        time_diff = (datos_boya_proc["date"][i] - datos_boya_proc["date"][i-1]).total_seconds()
        time_jump=(time_diff+loosed_minutes*60)/60/5
        loosed_minutes = (time_diff+loosed_minutes*60)%5
        
        # Place the same label values along the free slot equally distributed max 1 hour 
        same_time_jump = int(min(int(time_jump),12)/num_same_values)
        if (same_time_jump!=0): # If we have more points than slots within an hour dont trust the data
            for l in range(num_same_values):
                full_dataset[full_data_index+l*same_time_jump] = real_dataset[i+(l-num_same_values-1)]
        
    time_jump_round = int(time_jump)
    full_data_index += time_jump_round
    full_dataset[full_data_index] = real_dataset[i]
    i+=1


full_dataset_t = np.transpose(full_dataset)
y = full_dataset_t[1]
x = range(np.shape(full_dataset_t)[1])
plt.scatter(x,y)
plt.xlabel("Time (5min/step)")
plt.ylabel("Value")
plt.show()

In [ ]:
# Interpolate to fill the full_dataset time series (https://stackoverflow.com/questions/6518811/interpolate-nan-values-in-a-numpy-array)
def nan_helper(y):
    """Helper to handle indices and logical indices of NaNs.

    Input:
        - y, 1d numpy array with possible NaNs
    Output:
        - nans, logical indices of NaNs
        - index, a function, with signature indices= index(logical_indices),
          to convert logical indices of NaNs to 'equivalent' indices
    Example:
        >>> # linear interpolation of NaNs
        >>> nans, x= nan_helper(y)
        >>> y[nans]= np.interp(x(nans), x(~nans), y[~nans])
    """

    return np.isnan(y), lambda z: z.nonzero()[0]

full_dataset_interp = copy.deepcopy(full_dataset)
full_dataset_t = np.transpose(full_dataset_interp)
for i in range(np.shape(full_dataset_t)[0]):
    nans, x= nan_helper(full_dataset_t[i])
    if(i == 0): # The 0 feature is the month and "cant" be decimal
        #print("Flooring the month")
        full_dataset_t[i][nans]= np.floor(np.interp(x(nans), x(~nans), full_dataset_t[i][~nans]))
    else:
        #print("Not flooring the rest")
        full_dataset_t[i][nans]= np.interp(x(nans), x(~nans), full_dataset_t[i][~nans])
                                  
full_dataset_interp = np.transpose(full_dataset_t)

full_dataset_t = np.transpose(full_dataset_interp)
y= full_dataset_t[1]#[106000:108000]
x = range(np.shape(full_dataset_t)[1])#(2000)
plt.xlabel("Time (5min/step)")
plt.ylabel("Value")
plt.scatter(x,y)
plt.show()

In [ ]:
# Normalize the data
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

# the scaler object (model)
scaler = StandardScaler() # fit and transform the data
full_dataset_interp_norm = scaler.fit_transform(full_dataset_interp)

print(np.shape(full_dataset_interp_norm))

In [ ]:
# Export dataset to csv
#pd.DataFrame(full_dataset_interp_norm).to_csv('sample.csv')

In [ ]:
# Lets create dataset for training and testing
# First create a time column list for the time
import datetime
time_column = []
for i in range(total_time_steps): #total_time_steps = int((end_time-start_time).total_seconds() / 60 / 5)
    time_column.append(start_time + datetime.timedelta(minutes=5*i))

# Create input/output samples
def split_sequences(sequences, sequences_interp, time_seq, in_window, step, pred_window, in_freq, out_freq):
    X_train, y_train, time_train, time_train_out = list(), list(), list(), list()
    X_test, y_test, time_test, time_test_out = list(), list(), list(), list()
    i=0; test_flag = 0; test_date = datetime.datetime(2014, 5, 1) # test date in yyyy/mm/dd format. This day starts testing
    while(i<len(sequences)):
        # find the end of this pattern
        end_ix = i + in_window
        end_out_ix = end_ix + pred_window
        
        # check if we are beyond the dataset
        if end_out_ix > len(sequences):
            break
               
        # gather input and output parts of the pattern
        in_seq, out_seq = sequences_interp[i:end_ix], sequences_interp[end_ix:end_out_ix, -1]
        time_seq = time_column[i:end_ix]
        time_seq_out = time_column[end_ix:end_out_ix]
        
        # Lets fill things
        X_tmp, y_tmp = list(), list()
        # Fill with mean
        for k in range(1,len(in_seq)+1):
            if (k%in_freq) == 0:
                X_tmp.append(np.mean(in_seq[(k-in_freq):k], axis = 0))   
        for k in range(1,len(out_seq)+1):   
            if (k%out_freq) == 0:
                y_tmp.append(np.mean(out_seq[(k-out_freq):k], axis = 0))
        if(time_column[i] < test_date): #Train
            X_train.append(X_tmp)
            y_train.append(y_tmp)
            time_train.append(time_seq[in_freq::in_freq])
            time_train_out.append(time_seq_out[0:-out_freq:out_freq]) 
        else: #Test
            if (test_flag == 0):
                print("Changed year at index: ", i)
                changed_year_index = i
                test_flag = 1
            X_test.append(X_tmp)
            y_test.append(y_tmp)
            time_test.append(time_seq[0::in_freq])
            time_test_out.append(time_seq_out[0::out_freq]) 
        
        # Next step
        i+=step
        
    return np.array(X_train), np.array(y_train), time_train, np.array(X_test), np.array(y_test), time_test, time_train_out, time_test_out, changed_year_index
 
step = 12; in_steps = 8064; pred_steps = 8064; in_freq = 288; out_freq=288
X_train, y_train, time_train, X_test, y_test, time_test, time_train_out, time_test_out, changed_year_index = split_sequences(full_dataset, full_dataset_interp_norm, time_column, in_steps, step, pred_steps, in_freq, out_freq)

in_window = int(in_steps/in_freq); pred_window = int(pred_steps/out_freq)
print(np.shape(X_train))
print(np.shape(y_train))
print(np.shape(X_test))
print(np.shape(y_test))
print("The in window is: ", in_window)
print("The out window is: ", pred_window)

In [ ]:
# Import data aug and perform it (https://github.com/uchidalab/time_series_augmentation/)
import sys
sys.path.insert(1, '/home/raul/repos/time_series_augmentation') #(https://stackoverflow.com/questions/4383571/importing-files-from-different-folder)
from utils.input_data import read_data_sets
import utils.datasets as ds
import utils.augmentation as aug
import utils.helper as hlp

# Lets data aug
X_train_jitter = aug.jitter(X_train)
X_train_jitter_2 = aug.jitter(X_train, sigma=0.1)
X_train_jitter_3 = aug.jitter(X_train, sigma=0.2)
X_train_scale = aug.scaling(X_train, sigma=0.6)
X_train_scale_2 = aug.scaling(X_train, sigma=1)
X_train_magwarp = aug.magnitude_warp(X_train, sigma=1, knot=4)
X_train_magwarp_2 = aug.magnitude_warp(X_train, sigma=0.6, knot=14)
X_train_timewarp = aug.time_warp(X_train, sigma=0.4, knot=4)
X_train_timewarp_2 = aug.time_warp(X_train, sigma=0.2, knot=14)

# Add to train dataset
X_train_aug = np.concatenate((X_train, X_train_jitter, X_train_jitter_2, X_train_jitter_3, X_train_scale, X_train_scale_2, X_train_magwarp, X_train_magwarp_2, X_train_timewarp, X_train_timewarp_2))
y_train_aug = np.concatenate((y_train, y_train, y_train, y_train, y_train , y_train, y_train, y_train, y_train, y_train))

print(np.shape(X_train_aug))
print(np.shape(y_train_aug))

In [ ]:
# Plot data aug example
hlp.plot1d(X_train[0], X_train_jitter[0])
hlp.plot1d(X_train[0], X_train_jitter_2[0])
hlp.plot1d(X_train[0], X_train_jitter_3[0])
hlp.plot1d(X_train[0], X_train_scale[0])
hlp.plot1d(X_train[0], X_train_scale_2[0])
hlp.plot1d(X_train[0], X_train_magwarp[0])
hlp.plot1d(X_train[0], X_train_magwarp_2[0])
hlp.plot1d(X_train[0], X_train_timewarp[0])
hlp.plot1d(X_train[0], X_train_timewarp_2[0])

In [ ]:
# Train the model
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
import keras

# First define num features
n_features = X_train_aug.shape[2]

if TRAIN==1:
    # define model
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(in_window, n_features)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(pred_window))
    optimizer = keras.optimizers.Adam(lr=0.000005)
    model.compile(optimizer=optimizer, loss='mse', metrics=['accuracy'])

    # fit model
    batch_size = 100
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30) # early stop for validation
    history = model.fit(X_train_aug, y_train_aug, batch_size=batch_size , epochs=1000, validation_data=(X_test,y_test), callbacks=[es])

    # Save the model
    model.save('modelo.h5')
else:
    model = load_model('../models/model.h5')
    # summarize model.
    model.summary()
    

In [ ]:
# Now plot full trajectory every 5 min
# Plot sequence
y_test_pred = model.predict(X_test)
y_train_pred = model.predict(X_train)

In [ ]:
# Lets define some useful functions
def revert_out_norm(y_norm):
    # This function takes the output of the network and perform the inverse normalization operation
    y_raw = list()
    for i in range(len(y_norm)):
        y_raw_tmp = list()
        for j in range(len(y_norm[i])):
            y_raw_tmp.append(scaler.inverse_transform(np.pad([y_norm[i,:][j]], (n_features-1,0)).reshape(1, -1))[0,n_features-1])
        y_raw.append(y_raw_tmp)
    return y_raw

def remove_duplicates(sequences, time_info, index):
    # This function remove data that is duplicated in time from sequences
    # Sequences is a NxM array of arrays. Where N is the num os samples and M is the number of pred days.
    # time_info is the time info for each day
    # Index is the day index we want to use
    pred_steps = index*24; # Number of predicted steps. Before this steps we have no info.
    clean_seq = []; clean_time = [];
    # Init to first step 
    clean_seq.append(sequences[0][0])
    clean_time.append(time_info[0][0])
    for i in range(pred_steps): #The first index days steps are empty
        clean_seq.append(sequences[0][0])
        clean_time.append(time_info[i][0])

    # Continue with the rest
    for i in range(1, (len(sequences)-pred_steps-1)):
        if(time_info[i][index]-clean_time[-1]).total_seconds()>0:
            clean_seq.append(sequences[i][index])
            clean_time.append(time_info[i][index])
    return clean_seq, clean_time

def interpolate(inp, fi):
    i, f = int(fi // 1), fi % 1  # Split floating-point index into whole & fractional parts.
    j = i+1 if f > 0 else i  # Avoid index error.
    return (1-f) * inp[i] + f * inp[j]

In [ ]:
# Revert normalization
y_train_raw = revert_out_norm(y_train)
y_test_raw = revert_out_norm(y_test)
y_train_pred_raw = revert_out_norm(y_train_pred)
y_test_pred_raw = revert_out_norm(y_test_pred)

index = 0

# Clean duplicates. The architecture give us a 28 day sequence for each 28 day sequence we have. So we have a lot of days duplicated.
y_train_clean, clean_train_time = remove_duplicates(y_train_raw, time_train_out, index)
y_train_pred_clean, clean_train_pred_time = remove_duplicates(y_train_pred_raw, time_train_out, index)

y_test_clean, clean_test_time = remove_duplicates(y_test_raw, time_test_out, index)
y_test_pred_clean, clean_test_pred_time = remove_duplicates(y_test_pred_raw, time_test_out, index)

print(np.shape(y_test_clean))
print(np.shape(y_test_pred_clean))

########## Plot pred_train vs train #######
if(len(y_train_pred_clean)!=len(y_train_clean)):
    print("Warning plotting two traj with different lengths in same place")

plt.plot(y_train_clean, label="true")   
plt.plot(y_train_pred_clean, label="estimated")
plt.legend()
plt.xlabel("Time step (60min/step)")
plt.ylabel("Ficocianinas (ug/L)")
plt.savefig("pred-train.pdf")
plt.show()


########## Plot pred_train vs full train trajectory #########
full_base_traj = full_dataset_interp[in_steps:changed_year_index,-1]

# Pred
delta = (len(y_train_pred_clean)-1) / (len(full_base_traj)-1)
outp = [interpolate(y_train_pred_clean, i*delta) for i in range(len(full_base_traj))]
pad_zeros = np.zeros(in_steps+int(pred_steps/2)) # The first in_steps are not predicted, then the next step are moved +-1/2 of the average due to being the average of pred_window
outp_pred = np.concatenate((pad_zeros, outp))

if(len(outp_pred)!=len(full_dataset_interp[:changed_year_index,-1])):
    print("Warning plotting two traj with different lengths in same place")
    
plt.ylim([0, 40])
plt.plot(full_dataset_interp[:changed_year_index,-1], label="true")
plt.plot(outp_pred, label="estimated")
plt.legend()
plt.xlabel("Time step (60min/step)")
plt.ylabel("Ficocianinas (ug/L)")
plt.savefig("pred-train-full.pdf")
plt.show()



########## Plot pred_test vs test #######

if(len(y_test_pred_clean)!=len(y_test_clean)):
    print("Warning plotting two traj with different lengths in same place")

plt.plot(y_test_clean, label="true")
plt.plot(y_test_pred_clean, label="estimated")
plt.legend()
plt.xlabel("Time step (60min/step)")
plt.ylabel("Ficocianinas (ug/L)")
plt.savefig("pred-test.pdf")
plt.show()


########## Plot pred_test vs full test trajectory #########
full_base_traj = full_dataset_interp[in_steps+changed_year_index:,-1]

# Pred
delta = (len(y_test_pred_clean)-1) / (len(full_base_traj)-1)
outp = [interpolate(y_test_pred_clean, i*delta) for i in range(len(full_base_traj))]

if(len(outp)!=len(full_dataset_interp[changed_year_index:,-1])):
    print("Warning plotting two traj with different lengths in same place")
    
plt.ylim([0, 40])
plt.plot(full_dataset_interp[changed_year_index:,-1], label="true")
plt.plot(outp, label="estimated")
plt.legend()
plt.xlabel("Time step (60min/step)")
plt.ylabel("Ficocianinas (ug/L)")
plt.savefig("pred-test-full.pdf")
plt.show()

print(np.shape(y_test_pred_clean))
print(np.shape(y_test_clean))

In [ ]:
y_test_pred_clean_memory = list()
clean_test_pred_time_memory = list()
y_test_clean, clean_train_pred_time = remove_duplicates(y_test_raw, time_test_out, 0)
for i in range(np.shape(y_test_raw)[1]):
    # The result is aligned because each result has an absolute time value
    y_test_pred_clean, clean_test_pred_time = remove_duplicates(y_test_pred_raw, time_test_out, i) 
    y_test__clean, clean_test_pred_time = remove_duplicates(y_test_raw, time_test_out, i)
    
    if(len(y_test_pred_clean)!=len(y_test_clean)):
        print("Warning plotting two traj with different lengths in same place")
    
    y_test_pred_clean_memory.append(y_test_pred_clean)
    clean_test_pred_time_memory.append(clean_test_pred_time)
    
    plt.plot(y_test_clean, label="true")
    plt.plot(y_test_pred_clean, label="estimated")
    plt.xlabel("Time step (60min/step)")
    plt.ylabel("Ficocianinas (ug/L)")
    plt.savefig("pred_"+str(i)+".pdf")
    plt.legend()
    plt.show()

In [ ]:
print(np.shape(y_test_pred_clean))
print(np.shape(y_test_clean))

In [ ]:
np.shape(y_test_raw)

In [ ]:
# Compute percentage error 
y_test_clean = np.array(y_test_clean)
percentage_hist = []

for i in range(len(y_test_pred_clean_memory)):
    y_test_pred_clean_memory[i] = np.array(y_test_pred_clean_memory[i])
    # Compute the percentage
    percentage = 100-abs(y_test_clean-y_test_pred_clean_memory[i])/y_test_clean*100
    percentage = sum(percentage)/len(percentage)
    percentage_hist.append(percentage)

plt.plot(percentage_hist)
plt.ylim([50, 100])
plt.xlabel("Prediction Days")
plt.ylabel("Precision %")
plt.savefig("precision.pdf")
plt.show()


In [ ]:
# Compute discrete error
threshold_bin = (max(y_test_clean)-min(y_test_clean))/5
start = min(y_test_clean)
y_test_bin = np.digitize(y_test_clean,[start, start+threshold_bin, start+threshold_bin*2, start+threshold_bin*3, start+threshold_bin*4])
percentage_hist_bin = []

for i in range(len(y_test_pred_clean_memory)):
    y_test_pred_bin = np.digitize(y_test_pred_clean_memory[i],[start, start+threshold_bin, start+threshold_bin*2, start+threshold_bin*3, start+threshold_bin*4])
    # Compute the percentage
    percentage = 100-abs(y_test_bin-y_test_pred_bin)/y_test_bin*100
    percentage = sum(percentage)/len(percentage)
    percentage_hist_bin.append(percentage)

plt.plot(percentage_hist_bin)
plt.xlabel("Prediction Days")
plt.ylabel("Precision %")
plt.savefig("precision-bin.pdf")
plt.show()


In [ ]:
################################
#permutation importance session#
################################
from sklearn.metrics import mean_squared_error
import IPython

randsample = 10

errreal = mean_squared_error(y_test_raw, y_test_pred_raw)

# For each sample shape (1,in_window,n_features) one feature is replaced for random values 
permutsample = np.zeros((randsample, in_window, n_features))

for trying in range(randsample): 
    randval = np.zeros((len(y_test), in_window)) # For each output we create a random input with input size
    print(" ")
    print("Iteracion: ", trying, end = '')
    for i in range(len(y_test)):
        randval[i] = np.random.uniform(-1,1, in_window)
    
    for i in range(np.shape(X_test)[2]):
        print(".", end = '')
        permutinput = np.zeros(np.shape(X_test))
        permutinput[:] = X_test
        permutinput[:,:,i] = randval
        y_pred_rand = model.predict(permutinput)
        y_pred_rand_raw = revert_out_norm(y_pred_rand)
        err = mean_squared_error(y_test_raw, y_pred_rand_raw)
        permutsample[trying, i] = err


In [ ]:
#print mean and standard deviation of error
errperformance = np.zeros((np.shape(X_test)[2], 2))
for i in range(np.shape(X_test)[2]):
    errperformance[i,0] = np.mean(permutsample[:,i])
    errperformance[i,1] = np.std(permutsample[:,i])
errperformance[:,0] = errreal - errperformance[:,0]

# Full
param_dict = ["date", "D", "T", "C", "C25", "DO", "PH", "ORP", "CHLA_IVF", "CDOMt", "FC_IVF"]

# As the boat
param_dict = ["date", "T", "DO", "PH", "FC_IVF"] 

for i in range(len(param_dict)):
    print(param_dict[i], errperformance[i]*1000)

In [ ]:
y_pos = range(len(errperformance))
param_dict_plot = ('date', 'D','T','C','C25','DO','PH','ORP','CHLA','CDOM', 'FC')
param_dict_plot = ('date', 'T', 'DO', 'PH', 'FC_IVF')
plt.bar(y_pos, -errperformance[:,0])
plt.xticks(y_pos, param_dict_plot)
plt.savefig("input-weight.pdf")
plt.show()

In [ ]:
errperformance[:,0]

In [ ]:
np.shape(X_test)[1]

In [ ]:
# Save things in text 
with open('data.txt', 'w') as f:
    f.write("Error/desviacion introducido por cada feature ")
    f.write("\n")
    for i in range(len(errperformance)):
        f.write(param_dict_plot[i])
        f.write(" ")
        f.write(str(errperformance[i][0]*1000))
        f.write(" ")
        f.write(str(errperformance[i][1]*1000))
        f.write("\n")
    f.write("Precision Continua en funcion del dia\n")
    for i in range(len(percentage_hist)):
        f.write(str(percentage_hist[i]))
        f.write(" ")
    f.write("Precision Discreta en funcion del dia\n")
    for i in range(len(percentage_hist_bin)):
        f.write(str(percentage_hist_bin[i]))
        f.write(" ")

In [ ]:
if TRAIN == 1:
    # list all data in history
    print(history.history.keys())
    # summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig("accuracy.pdf")
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig("loss.pdf")
    plt.show()